<a href="https://colab.research.google.com/github/AndreyPro39/AndreyPro39/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22test_scoring_(4)_(2)_(3)_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import sklearn
import imblearn
import matplotlib as plt
from matplotlib import pyplot as plt
!pip install catboost
!pip install xgboost
!pip install lightgbm
!pip install category_encoders
!pip install phik
import phik
from sklearn.model_selection import train_test_split
import seaborn as sns
import datetime
from datetime import datetime
from phik.report import plot_correlation_matrix
from sklearn.preprocessing import LabelEncoder
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
!pip install shap -q
import shap
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from scipy.stats import zscore
from phik.report import plot_correlation_matrix
from phik import phik_matrix
from phik.binning import bin_data


In [4]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/shift_ml_2025_train.csv'
df_train = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-3780925240.py:4: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(file_path)


In [5]:
df_train.head()

,id,сумма_займа,срок_займа,процентная_ставка,аннуитет,рейтинг,допрейтинг,профессия_заемщика,стаж,владение_жильем,...,процент_счетов_прев_75_лимита,кол-во_публ_банкротств,кол-во_залогов,кредитный_лимит,кредитный_баланс_без_ипотеки,лимит_по_картам,лимит_по_аннуитетным_счетам,кредитный_баланс_по_возоб_счетам,особая_ситуация,тип_предоставления_кредита
0,68355089,1235000.0,3 года,11.99,41014.0,В,В1,инженер,10+ лет,ИПОТЕКА,...,7.7,0.0,0.0,15700850.0,1973750.0,3965000.0,1233350.0,NaN,Нет,Наличные
1,68341763,1000000.0,5 лет,10.78,21633.0,Б,Б4,водитель грузовика,10+ лет,ИПОТЕКА,...,50.0,0.0,0.0,10920900.0,934800.0,310000.0,743850.0,NaN,Нет,Наличные
2,68426831,597500.0,3 года,13.44,20259.0,В,В3,ветеринарный техник,4 года,АРЕНДА,...,100.0,0.0,0.0,845000.0,639900.0,470000.0,200000.0,NaN,Нет,Наличные
3,68476668,1000000.0,3 года,9.17,31879.0,Б,Б2,вице-президент операций по набору персонала,10+ лет,ИПОТЕКА,...,100.0,0.0,0.0,19442600.0,5838100.0,1575000.0,2322600.0,NaN,Нет,Наличные
4,67275481,1000000.0,3 года,8.49,31563.0,Б,Б1,дорожному водителю,10+ лет,ИПОТЕКА,...,0.0,0.0,0.0,9669500.0,1396850.0,725000.0,1807200.0,NaN,Нет,Наличные


In [6]:
file_path_2 = '/content/drive/My Drive/shift_ml_2025_test.csv'

In [7]:
df_test = pd.read_csv(file_path_2)

In [8]:
df_test.head()

,id,сумма_займа,срок_займа,процентная_ставка,аннуитет,рейтинг,допрейтинг,профессия_заемщика,стаж,владение_жильем,...,процент_счетов_прев_75_лимита,кол-во_публ_банкротств,кол-во_залогов,кредитный_лимит,кредитный_баланс_без_ипотеки,лимит_по_картам,лимит_по_аннуитетным_счетам,кредитный_баланс_по_возоб_счетам,особая_ситуация,тип_предоставления_кредита
0,85540387,450000.0,3 года,9.49,14413.0,Б,Б2,обслуживание клиентов,10+ лет,ИПОТЕКА,...,75.0,0.0,0.0,4282850.0,1180600.0,725000.0,1022000.0,NaN,Нет,Наличные
1,28112500,400000.0,3 года,6.03,12174.5,А,А1,помощник по правовым вопросам,5 лет,АРЕНДА,...,57.1,0.0,0.0,3340900.0,2381050.0,1260000.0,1548550.0,NaN,Нет,Наличные
2,65731570,1250000.0,3 года,12.05,41548.0,В,В1,специалист по анализу кредитоспособности,5 лет,ИПОТЕКА,...,16.7,0.0,0.0,11350200.0,1980650.0,1690000.0,1804450.0,NaN,Нет,Наличные
3,65874747,977500.0,5 лет,20.99,26439.5,Д,Д5,специальный специалист,3 года,ИПОТЕКА,...,66.7,1.0,0.0,9976550.0,1678600.0,470000.0,2139950.0,NaN,Нет,Наличные
4,57893355,520000.0,3 года,18.25,18865.0,Д,Д1,руководитель районного проекта,3 года,АРЕНДА,...,66.7,0.0,0.0,1953950.0,1272750.0,1125000.0,603950.0,NaN,Нет,Наличные


Выделим целевой признак.

In [9]:
target = df_train['итоговый_статус_займа']

Данные загружены. Изучим общую информацию о данных.

In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1210779 entries, 0 to 1210778
Columns: 109 entries, id to тип_предоставления_кредита
dtypes: float64(84), int64(6), object(19)
memory usage: 1006.9+ MB


Имеем в тренировочной выборке 1210779 строк и 109 столбцов весом больше 1 Гб. Попробуем уменьшить размерность данных.

In [11]:
def optimize_data(df):
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    return df

In [12]:
df_train = optimize_data(df_train)

In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1210779 entries, 0 to 1210778
Columns: 109 entries, id to тип_предоставления_кредита
dtypes: float32(74), float64(10), int32(2), int8(4), object(19)
memory usage: 623.5+ MB


Видим, что размерность снизилась с 1Гб до 628 мегабайт. Проделаем то же самое с тестовой выборкой.

In [14]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134531 entries, 0 to 134530
Columns: 108 entries, id to тип_предоставления_кредита
dtypes: float64(85), int64(5), object(18)
memory usage: 110.9+ MB


In [15]:
df_test = optimize_data(df_test)

In [16]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134531 entries, 0 to 134530
Columns: 108 entries, id to тип_предоставления_кредита
dtypes: float32(76), float64(9), int32(2), int8(3), object(18)
memory usage: 68.1+ MB


Обьединим трейн и тест.

In [ ]:
def columns_unique(data):
    for col in data.columns:
        plt.figure(figsize=(10, 5))
        data[col].hist(grid=True)
        plt.xlabel(col)
        plt.ylabel('Количество')
        plt.title(f'Распределение признака: {col}')
        plt.show()

In [17]:
df_test['итоговый_статус_займа'] = -1
data = pd.concat([df_train, df_test], ignore_index=True)
data.head()

,id,сумма_займа,срок_займа,процентная_ставка,аннуитет,рейтинг,допрейтинг,профессия_заемщика,стаж,владение_жильем,...,процент_счетов_прев_75_лимита,кол-во_публ_банкротств,кол-во_залогов,кредитный_лимит,кредитный_баланс_без_ипотеки,лимит_по_картам,лимит_по_аннуитетным_счетам,кредитный_баланс_по_возоб_счетам,особая_ситуация,тип_предоставления_кредита
0,68355089,1235000.0,3 года,11.99,41014.0,В,В1,инженер,10+ лет,ИПОТЕКА,...,7.7,0.0,0.0,15700850.0,1973750.0,3965000.0,1233350.0,NaN,Нет,Наличные
1,68341763,1000000.0,5 лет,10.78,21633.0,Б,Б4,водитель грузовика,10+ лет,ИПОТЕКА,...,50.0,0.0,0.0,10920900.0,934800.0,310000.0,743850.0,NaN,Нет,Наличные
2,68426831,597500.0,3 года,13.44,20259.0,В,В3,ветеринарный техник,4 года,АРЕНДА,...,100.0,0.0,0.0,845000.0,639900.0,470000.0,200000.0,NaN,Нет,Наличные
3,68476668,1000000.0,3 года,9.17,31879.0,Б,Б2,вице-президент операций по набору персонала,10+ лет,ИПОТЕКА,...,100.0,0.0,0.0,19442600.0,5838100.0,1575000.0,2322600.0,NaN,Нет,Наличные
4,67275481,1000000.0,3 года,8.49,31563.0,Б,Б1,дорожному водителю,10+ лет,ИПОТЕКА,...,0.0,0.0,0.0,9669500.0,1396850.0,725000.0,1807200.0,NaN,Нет,Наличные


Найдем пропуски и явные дубликаты в таблицах.

In [18]:
pd.set_option('display.max_colwidth', 100)

In [19]:
data.duplicated().sum()

np.int64(0)

Дубликатов нет.

Выведем все названия столбцов трейна на экран.

Уберем ограничение максимального количества отображения колонок в одной ячейке.

In [20]:
pd.set_option('display.max_column', None)
pd.set_option('display.expand_frame_repr', False)
sorted_cols = data.columns.sort_values()
print(sorted_cols.to_list())

['id', 'аннуитет', 'верхний_порог_рейтинга_заемщика', 'владение_жильем', 'возвраты_с_учетом_мпл', 'годовой_доход', 'грейд_на_детерминаторе', 'дата_первого_займа', 'дата_следующей_выплаты', 'допрейтинг', 'индекс_проживания', 'итоговый_статус_займа', 'код_политики', 'кол-во_активных_возобновляемых_счетов', 'кол-во_активных_карт', 'кол-во_аннуитетных_счетов', 'кол-во_аннуитетных_счетов_за_2_года', 'кол-во_аннуитетных_счетов_за_год', 'кол-во_взысканий_за_последний_год', 'кол-во_возоб_счетов_за_2_года', 'кол-во_возоб_счетов_за_год', 'кол-во_возобновляемых_счетов', 'кол-во_возобновляемых_счетов_с_балансом_более_0', 'кол-во_залогов', 'кол-во_записей_в_общедоступных_документах', 'кол-во_заявок_за_полгода', 'кол-во_заявок_на_кредит_за_год', 'кол-во_ипотек', 'кол-во_карт', 'кол-во_карт_без_нарушений', 'кол-во_мес_с_последней_задолженности_по_возобновляемому_счету', 'кол-во_мес_с_последней_задолженности_по_карте', 'кол-во_мес_с_последней_заявки', 'кол-во_месяцев_с_первого_аннуитетного_счета', 'ко

На первый взгляд видно, что очень много признаков и лишних тоже много.\
Если взять из практики модель кредитного скоринга, то можно сразу убрать лишние признаки,\
такие как грейд_на_детерминаторе например и количество_возобновляемых_счетов, доп рейтинг и так далее.

Посмотрим на уникальные значения каждой колонки в отдельности.

In [21]:
for col in data.columns:
    columns_unique = data[col].sort_values().unique()
    print(f'уникальные значения столбца {col}: {columns_unique}')

уникальные значения столбца id: [    54734     55716     55742 ... 145623694 145627026 145636374]
уникальные значения столбца сумма_займа: [  25000.   35000.   36250. ... 1997500. 1998750. 2000000.]
уникальные значения столбца срок_займа: ['3 года' '5 лет']
уникальные значения столбца процентная_ставка: [ 5.31  5.32  5.42  5.79  5.93  5.99  6.    6.03  6.07  6.08  6.11  6.17
  6.19  6.24  6.39  6.46  6.49  6.54  6.62  6.67  6.68  6.71  6.72  6.76
  6.83  6.89  6.91  6.92  6.97  6.99  7.02  7.05  7.07  7.12  7.14  7.21
  7.24  7.26  7.29  7.34  7.35  7.37  7.39  7.4   7.42  7.43  7.46  7.49
  7.51  7.56  7.59  7.62  7.66  7.68  7.69  7.74  7.75  7.84  7.88  7.89
  7.9   7.91  7.96  7.97  7.99  8.    8.07  8.08  8.18  8.19  8.24  8.32
  8.38  8.39  8.46  8.49  8.59  8.6   8.63  8.67  8.7   8.81  8.88  8.9
  8.94  8.99  9.01  9.07  9.16  9.17  9.2   9.25  9.32  9.33  9.38  9.43
  9.44  9.45  9.49  9.51  9.58  9.62  9.63  9.64  9.67  9.7   9.71  9.75
  9.76  9.8   9.83  9.88  9.91  9.92  9

**Вывод: Данные требуют обработки. Есть пропуски, есть лог запись данных, требующая нормализации. Некоторые столбцы не содержат данных,\
либо содержат всего одно значение. Удалим такие столбцы.**

На основе анализа списка параметров и согласно интернету, следующие колонки маловероятно влияют на модель кредитного скоринга или имеют минимальное влияние:\
Технические параметры системы: код_политики, грейд_на_детерминаторе, дата_следующей_выплаты

Избыточные или дублирующие показатели:
совокупный_доход_заемщиков (дублируется с годовой_доход),
совокупный_пдн_заемщиков (дублируется с пдн),
кол-во_заявок_за_полгода (частично дублируется с другими метриками активности).

Показатели с низкой предсказательной силой:
кол-во_публичных_банкротств (редко встречается в данных),
кол-во_залогов (не всегда коррелирует с платежеспособностью),
коэфф_акционных_зачислений_в_остатке (не связан напрямую с платежеспособностью).

Промежуточные расчетные показатели:
оборотный_баланс,
коэфф_загрузки_возобновляемого_счета,
возвраты_с_учетом_мпл.

Показатели с высокой корреляцией с другими метриками:
кол-во_открытых_счетов_за_полгода (коррелирует с общей активностью),
кол-во_финансовых_операций (частично дублирует активность),
процент_счетов_прев_75_лимита (уже учтен в других метриках лимитов).
Удалим эти признаки для дальнейшей оптимизации данных для обучения модели.

In [22]:
data = data.drop(
    ['код_политики', 'непогашенная_сумма_из_тела_займов', 'грейд_на_детерминаторе', 'дата_следующей_выплаты', 'совокупный_доход_заемщиков',
'совокупный_пдн_заемщиков', 'кол-во_заявок_за_полгода', 'кол-во_публ_банкротств', 'кол-во_залогов', 'платежный_график',
'коэфф_акционных_зачислений_в_остатке', 'оборотный_баланс', 'коэфф_загрузки_возобновляемого_счета',
'возвраты_с_учетом_мпл', 'кол-во_открытых_счетов_за_полгода', 'кол-во_финансовых_операций', 'процент_счетов_прев_75_лимита'], axis=1
)

In [23]:
data.shape

(1345310, 92)

Минусовали по 15 лишних столбцов обьединенного датафрейма.

Посмотрим на пропуски после удаления лишних признаков.

In [24]:
pd.set_option('display.max_rows', None)
data.isna().sum().sort_values(ascending=False)

,0
кредитный_баланс_по_возоб_счетам,1326681
совокупный_статус_подтверждения_доходов_заемщиков,1319715
кол-во_месяцев_с_последнего_займа,1116755
кол-во_мес_с_последней_задолженности_по_карте,1026290
кол-во_месяцев_с_последнего_нарушения,991560
кол-во_мес_с_последней_задолженности_по_возобновляемому_счету,895348
соотношение_сумм_текущего_баланса_к_лимиту_по_аннуитетным_счетам,880294
кол-во_месяцев_с_посл_аннуитетного_счета,821928
соотношение_баланса_к_лимиту_общее,807765
кол-во_заявок_на_кредит_за_год,807713


Обработаем пропуски. Такое большое количество пропусков в данных столбцах не означает что данные нужно удалять.\
Если удалим столько записей, будет большая потеря данных. Скорее нужно заменить данные пропуски значениями-заглушками, потомучто
модель должна работать со всеми типами клиентов. Тем более в нескольких колонках количество пропусков одинаково - значит и не было там никаких значений.

Например в столбце кредитный_баланс_по_возоб_счетам - более 98% пропусков,\
что связано скорее всего с тем, что клиенты не пользовались возобновляемыми счетами повторно.\
Поэтому заполним данные пропуски нулями. Аналогично сделаем с подобными пропусками.

In [25]:
columns = [
'кредитный_баланс_по_возоб_счетам', 'кол-во_месяцев_с_последнего_займа', 'кол-во_мес_с_последней_задолженности_по_карте',
'кол-во_месяцев_с_последнего_нарушения', 'кол-во_мес_с_последней_задолженности_по_возобновляемому_счету',
'соотношение_сумм_текущего_баланса_к_лимиту_по_аннуитетным_счетам', 'кол-во_месяцев_с_посл_аннуитетного_счета',
'соотношение_баланса_к_лимиту_общее', 'кол-во_заявок_на_кредит_за_год', 'кол-во_возоб_счетов_за_год',
'кол-во_возоб_счетов_за_2_года', 'кол-во_аннуитетных_счетов_за_год', 'кол-во_текущих_аннуитетных_счетов',
'макс_баланс_по_возоб_счетам', 'кол-во_финансовых_запросов', 'текущий_баланс_по_аннуитетным_счетам',
'кол-во_месяцев_с_последней_просрочки', 'процент_счетов_без_просрочек', 'средний_баланс_текущих_счетов',
'кол-во_месяцев_с_первого_возобновляемого_счета', 'кол-во_месяцев_с_последнего_возобновляемого_счета',
'кол-во_возобновляемых_счетов', 'кредитный_лимит', 'общий_лимит_по_возоб_счету',
'кол-во_счетов_за_посл_год',
'кол-во_счетов_с_акт_просрочкой_3_мес_за_2_года',
'кол-во_счетов_с_акт_просрочкой_1_мес',
'общая_сумма_на_счетах',
'кол-во_карт',
'лимит_по_аннуитетным_счетам',
'кол-во_аннуитетных_счетов',
'общая_сумма_взысканий',
'кол-во_открытых_возобновляемых_счетов',
'кол-во_счетов_с_просрочкой_4_мес',
'кол-во_месяцев_с_последнего_счета',
'кол-во_возобновляемых_счетов_с_балансом_более_0',
'кол-во_активных_возобновляемых_счетов',
'кол-во_активных_карт',
'соотношение_баланса_к_лимиту_по_картам',
'суммарная_доступная_сумма_займа_по_картам',
'кол-во_месяцев_с_последней_карты',
'кол-во_карт_без_нарушений',
'кол-во_счетов_без_нарушений',
'лимит_по_картам',
'кредитный_баланс_без_ипотеки',
'кол-во_открытых_счетов_за_2_года',
'кол-во_ипотек',
'пдн',
'кол-во_взысканий_за_последний_год',
'кол-во_списаний_за_год',
'пени_за_дефолт'
]
data[columns] = data[columns].fillna(0)

Нормализуем столбцы с лог записью.

In [26]:
numeric_cols = data.select_dtypes(include=['float64', 'float32', 'int64', 'int32']).columns
data[numeric_cols] = (data[numeric_cols] - data[numeric_cols].min()) / \
                  (data[numeric_cols].max() - data[numeric_cols].min())

In [33]:
data.head()

,id,сумма_займа,срок_займа,процентная_ставка,аннуитет,рейтинг,допрейтинг,профессия_заемщика,стаж,владение_жильем,годовой_доход,подтвержден_ли_доход,итоговый_статус_займа,рейтинг_OCR,цель_займа,кол_во_субсидий_за_год,индекс_проживания,регион,пдн,пос_стоп_фактор,юридический_статус,кол-во_просрочек_за_два_года,коэфф_невыплаченного_сумм_остатка,коэфф_исходного_платежа,дата_первого_займа,нижний_порог_рейтинга_заемщика,верхний_порог_рейтинга_заемщика,кол-во_месяцев_с_последней_просрочки,кол-во_месяцев_с_последнего_займа,кол-во_открытых_счетов,кол-во_записей_в_общедоступных_документах,кол-во_счетов,первоначальный_статус_займа,сумма_выплат_по_просрочкам,пени_за_дефолт,кол-во_взысканий_за_последний_год,кол-во_месяцев_с_последнего_нарушения,тип_займа,совокупный_статус_подтверждения_доходов_заемщиков,кол-во_текущих_просроченных_счетов,общая_сумма_взысканий,общая_сумма_на_счетах,кол-во_текущих_аннуитетных_счетов,кол-во_аннуитетных_счетов_за_год,кол-во_аннуитетных_счетов_за_2_года,кол-во_месяцев_с_посл_аннуитетного_счета,текущий_баланс_по_аннуитетным_счетам,соотношение_сумм_текущего_баланса_к_лимиту_по_аннуитетным_счетам,кол-во_возоб_счетов_за_год,кол-во_возоб_счетов_за_2_года,макс_баланс_по_возоб_счетам,соотношение_баланса_к_лимиту_общее,общий_лимит_по_возоб_счету,кол-во_финансовых_запросов,кол-во_заявок_на_кредит_за_год,кол-во_открытых_счетов_за_2_года,средний_баланс_текущих_счетов,суммарная_доступная_сумма_займа_по_картам,соотношение_баланса_к_лимиту_по_картам,кол-во_списаний_за_год,сумма_задолженности,кол-во_месяцев_с_первого_аннуитетного_счета,кол-во_месяцев_с_первого_возобновляемого_счета,кол-во_месяцев_с_последнего_возобновляемого_счета,кол-во_месяцев_с_последнего_счета,кол-во_ипотек,кол-во_месяцев_с_последней_карты,кол-во_мес_с_последней_задолженности_по_карте,кол-во_мес_с_последней_заявки,кол-во_мес_с_последней_задолженности_по_возобновляемому_счету,кол-во_счетов_с_просрочкой_4_мес,кол-во_активных_карт,кол-во_активных_возобновляемых_счетов,кол-во_карт_без_нарушений,кол-во_карт,кол-во_аннуитетных_счетов,кол-во_открытых_возобновляемых_счетов,кол-во_возобновляемых_счетов,кол-во_возобновляемых_счетов_с_балансом_более_0,кол-во_счетов_без_нарушений,кол-во_счетов_с_акт_просрочкой_4_мес_обн_2_мес,кол-во_счетов_с_акт_просрочкой_1_мес,кол-во_счетов_с_акт_просрочкой_3_мес_за_2_года,кол-во_счетов_за_посл_год,процент_счетов_без_просрочек,кредитный_лимит,кредитный_баланс_без_ипотеки,лимит_по_картам,лимит_по_аннуитетным_счетам,кредитный_баланс_по_возоб_счетам,особая_ситуация,тип_предоставления_кредита
0,0.469155,0.612658,3 года,0.260125,0.475450,В,В1,инженер,10+ лет,ИПОТЕКА,0.005910,Не подтвержден,0.5,4,мелкий_бизнес,0,0.574597,Новгородская область,0.01706,Нет,Нет,0.025641,NaN,0.767018,12-1999,0.409091,0.407240,0.026549,0.0,0.244444,0.0,0.206897,а,4.289932e-12,False,0.0,0.000000,Индивидуальный,NaN,0.0,0.0,0.025549,0.017544,0.0,0.019608,0.037182,0.010523,0.130824,0.071429,0.056604,0.008331,0.146465,0.01118,0.000000,0.089552,0.06250,0.010159,0.103284,0.079800,0.0,0.0,0.113113,0.225352,0.004566,0.006369,0.078431,0.003130,0.0,0.00,0.029703,0.000000,0.142857,0.079365,0.206349,0.242857,0.037736,0.240964,0.210938,0.111111,0.244444,0.0,0.0,0.000000,0.0625,0.974,0.031402,0.011583,0.071732,0.011735,0.0,Нет,Наличные
1,0.469063,0.493671,5 лет,0.213006,0.249420,Б,Б4,водитель грузовика,10+ лет,ИПОТЕКА,0.005728,Не подтвержден,0.5,2,улучшение_жилищных_условий,0,0.602823,Республика Татарстан,0.01178,Нет,Нет,0.000000,NaN,0.229990,8-2000,0.318182,0.316742,0.000000,0.0,0.066667,0.0,0.091954,а,4.289932e-12,0,0.0,0.000000,Совместный,Не подтвержден,0.0,0.0,0.023712,0.017544,0.0,0.078431,0.037182,0.006328,0.130824,0.000000,0.037736,0.002679,0.328283,0.00140,0.041667,0.014925,0.09375,0.033000,0.004888,0.164605,0.0,0.0,0.125125,0.215962,0.031963,0.044586,0.098039,0.158059,0.0,0.40,0.000000,0.000000,0.057143,0.047619,0.031746,0.057143,0.037736,0.048193,0.054688,0.066667,0.066667,0.0,0.0,0.000000,0.0000,1.000,0.021842,0.005486,0.005608,0.007078,0.0,Нет,Наличные
2,0.469648,

In [35]:
data.shape

(1345310, 92)

In [34]:
data.isna().sum()

,0
id,0
сумма_займа,0
срок_займа,0
процентная_ставка,0
аннуитет,0
рейтинг,0
допрейтинг,0
профессия_заемщика,85785
стаж,78511
владение_жильем,0


In [44]:
data['стаж'].value_counts()

,count
стаж,
10+ лет,442199
2 года,121743
< 1 года,108061
3 года,107597
1 год,88494
5 лет,84154
4 года,80556
6 лет,62733
8 лет,60701


расчитаем средний стаж для заполнения пропусков.

In [45]:
# Создаем словарь с данными
data = {
    'стаж': {
        '10+ лет': 442199,
        '2 года': 121743,
        '< 1 года': 108061,
        '3 года': 107597,
        '1 год': 88494,
        '5 лет': 84154,
        '4 года': 80556,
        '6 лет': 62733,
        '8 лет': 60701,
        '7 лет': 59624,
        '9 лет': 50937
    }
}

# Преобразуем категории стажа в числовые значения
def convert_experience(exp):
    if exp == '< 1 года':
        return 0.5
    elif exp == '10+ лет':
        return 10
    else:
        return int(exp.split()[0])

# Рассчитываем средний стаж
total_experience = 0
total_count = 0

for exp, count in data['стаж'].items():
    numeric_exp = convert_experience(exp)
    total_experience += numeric_exp * count
    total_count += count

average_experience = total_experience / total_count

print(f"Средний стаж: {average_experience:.2f} лет")


Средний стаж: 6.01 лет


In [ ]:
data['профессия_заемщика'] = data['профессия_заемщика'].fillna('нет профессии')
data['стаж'] = data['стаж'].fillna(average_experience)


In [28]:
cat_cols = data.select_dtypes(include=['object']).columns
cat_cols

Index(['срок_займа', 'рейтинг', 'допрейтинг', 'профессия_заемщика', 'стаж',
       'владение_жильем', 'подтвержден_ли_доход', 'цель_займа', 'регион',
       'пос_стоп_фактор', 'юридический_статус', 'дата_первого_займа',
       'первоначальный_статус_займа', 'пени_за_дефолт', 'тип_займа',
       'совокупный_статус_подтверждения_доходов_заемщиков', 'особая_ситуация',
       'тип_предоставления_кредита'],
      dtype='object')

In [32]:
data[numeric_cols].drop(['id', 'коэфф_невыплаченного_сумм_остатка'], axis=1).corr()

,сумма_займа,процентная_ставка,аннуитет,годовой_доход,итоговый_статус_займа,индекс_проживания,пдн,кол-во_просрочек_за_два_года,коэфф_исходного_платежа,нижний_порог_рейтинга_заемщика,верхний_порог_рейтинга_заемщика,кол-во_месяцев_с_последней_просрочки,кол-во_месяцев_с_последнего_займа,кол-во_открытых_счетов,кол-во_записей_в_общедоступных_документах,кол-во_счетов,сумма_выплат_по_просрочкам,кол-во_взысканий_за_последний_год,кол-во_месяцев_с_последнего_нарушения,кол-во_текущих_просроченных_счетов,общая_сумма_взысканий,общая_сумма_на_счетах,кол-во_текущих_аннуитетных_счетов,кол-во_аннуитетных_счетов_за_год,кол-во_аннуитетных_счетов_за_2_года,кол-во_месяцев_с_посл_аннуитетного_счета,текущий_баланс_по_аннуитетным_счетам,соотношение_сумм_текущего_баланса_к_лимиту_по_аннуитетным_счетам,кол-во_возоб_счетов_за_год,кол-во_возоб_счетов_за_2_года,макс_баланс_по_возоб_счетам,соотношение_баланса_к_лимиту_общее,общий_лимит_по_возоб_счету,кол-во_финансовых_запросов,кол-во_заявок_на_кредит_за_год,кол-во_открытых_счетов_за_2_года,средний_баланс_текущих_счетов,суммарная_доступная_сумма_займа_по_картам,соотношение_баланса_к_лимиту_по_картам,кол-во_списаний_за_год,сумма_задолженности,кол-во_месяцев_с_первого_аннуитетного_счета,кол-во_месяцев_с_первого_возобновляемого_счета,кол-во_месяцев_с_последнего_возобновляемого_счета,кол-во_месяцев_с_последнего_счета,кол-во_ипотек,кол-во_месяцев_с_последней_карты,кол-во_мес_с_последней_задолженности_по_карте,кол-во_мес_с_последней_заявки,кол-во_мес_с_последней_задолженности_по_возобновляемому_счету,кол-во_счетов_с_просрочкой_4_мес,кол-во_активных_карт,кол-во_активных_возобновляемых_счетов,кол-во_карт_без_нарушений,кол-во_карт,кол-во_аннуитетных_счетов,кол-во_открытых_возобновляемых_счетов,кол-во_возобновляемых_счетов,кол-во_возобновляемых_счетов_с_балансом_более_0,кол-во_счетов_без_нарушений,кол-во_счетов_с_акт_просрочкой_4_мес_обн_2_мес,кол-во_счетов_с_акт_просрочкой_1_мес,кол-во_счетов_с_акт_просрочкой_3_мес_за_2_года,кол-во_счетов_за_посл_год,процент_счетов_без_просрочек,кредитный_лимит,кредитный_баланс_без_ипотеки,лимит_по_картам,лимит_по_аннуитетным_счетам,кредитный_баланс_по_возоб_счетам
сумма_займа,1.000000,0.141673,0.953401,0.311794,0.045501,0.005101,0.031889,-0.003008,-0.000021,0.100984,0.100983,-0.020092,-0.084248,0.186200,-0.065915,0.208998,0.090032,-0.018491,-0.045692,0.001526,-0.003002,0.314814,0.016618,0.002160,0.028918,-0.002615,0.086011,-0.013559,-0.016937,-0.007143,0.187972,0.004938,0.314006,0.005396,0.011351,0.024949,0.232647,0.202847,0.076836,-0.000850,0.000428,0.123768,0.172482,0.056629,0.035803,0.227928,0.041928,-0.022573,-0.000149,-0.019421,-0.043943,0.201369,0.161789,0.219574,0.202931,0.089371,0.174757,0.178517,0.161285,0.189296,-0.000662,0.003216,-0.019223,-0.003659,0.091186,0.334135,0.278824,0.372148,0.204453,0.076142
процентная_ставка,0.141673,1.000000,0.153916,-0.072245,0.177784,-0.004951,0.146635,0.048311,0.000071,-0.405367,-0.405364,0.038604,0.061890,-0.004395,0.053035,-0.042425,0.075000,0.015304,0.058996,0.017235,0.002530,-0.075048,0.030412,0.119273,0.167463,-0.050915,0.032796,0.065331,0.092423,0.091632,-0.014883,0.094587,-0.149330,0.098098,0.119918,0.193279,-0.074888,-0.270413,0.236541,0.011386,0.005327,-0.071699,-0.110851,-0.097870,-0.116597,-0.074197,-0.078184,0.036376,-0.168946,0.035601,0.044926,0.032816,0.091672,-0.037325,-0.063927,0.019096,0.005171,-0.038074,0.091589,0.008733,0.006024,0.014398,0.027586,0.206526,0.008407,-0.106534,0.012263,-0.214502,0.003448,0.033309
аннуитет,0.953401,0.153916,1.000000,0.302499,0.036089,0.009486,0.035655,0.006469,-0.000241,0.050983,0.050983,-0.013311,-0.075379,0.176570,-0.054352,0.186025,0.098797,-0.014907,-0.035472,0.004311,-0.002461,0.286758,0.023773,0.016397,0.035109,0.002685,0.089561,-0.000478,0.003207,0.013760,0.191491,0.024985,0.291217,0.018094,0.028935,0.040542,0.208526,0.171847,0.099507,0.001260,0.001445,0.102874,0.156730,0.044139,0.028309,0.197561,0.030762,-0.015290,-0.014440,-0.011317,-0.035017,0.206252,0.170809,0.215506,0.193274,0.0

In [ ]:
Phik_matrix = data.phik_matrix()
plt.figure(figsize=(14, 12))
sns.heatmap(
    Phik_matrix,
    vmin=0, vmax=1,
    cmap='coolwarm',
    annot=False,
    fmt=".2f",
    linewidths=0.1
)

plt.title('Матрица корреляций phik_matrix'
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
sorted_cols = df_test.columns.sort_values()
print(sorted_cols.to_list())

Пропуски обработали, явных и неявных дубликатов не нашли. Построим графики признаков для наглядности\
и выявления выбросов и аномалий. Для этого создадим функцию с циклом для построения гистограмм.

In [ ]:
def columns_unique(data):
    for col in data.columns:
        plt.figure(figsize=(10, 5))
        data[col].hist(grid=True)
        plt.xlabel(col)
        plt.ylabel('Количество')
        plt.title(f'Распределение признака: {col}')
        plt.show()

In [ ]:
columns_unique(df_train)

In [ ]:
df_train.corr(method=spearman)

In [ ]:
columns_unique(df_test)

Для оптимизации признаков постороим матрицу корреляций по числовым значениям df.corr()